In [1]:
from hungarian import *
from generate_cost_matrix_from_user_input import *
from matrix_adjust import *
from output_assignment_list import *

Enter number of residents to be assigned to a hospital.

Enter names of hospitals with their respective capacities.

Enter the name of each resident and their ranking of each hospital in order.

In [2]:
ranks, residents, capacities = get_inputs()

Enter the number of residents to be assigned: 5
Enter hospital names and capacities (name1,capacity1,name2,capacity2,...):H1,2,H2,4,H3,3
Enter the name of resident 1: Bob
Enter hospital rankings for Bob (comma-separated):1,2,3
Enter the name of resident 2: Jim
Enter hospital rankings for Jim (comma-separated):3,1,2
Enter the name of resident 3: Dan
Enter hospital rankings for Dan (comma-separated):2,2,2
Enter the name of resident 4: Sally
Enter hospital rankings for Sally (comma-separated):3,2,1
Enter the name of resident 5: Mary
Enter hospital rankings for Mary (comma-separated):1,1,1


,H1,H2,H3
Bob,1,2,3
Jim,3,1,2
Dan,2,2,2
Sally,3,2,1
Mary,1,1,1


In [3]:
cost_matrix, hospital_names = matrix_with_capacities(ranks, capacities)

Use Hungarian algorithm to optimally assign each resident to a hospital according to their preferences.

In [4]:
ans_pos = hungarian_algorithm(cost_matrix.copy())

In [5]:
ans, ans_mat = ans_calculation(cost_matrix, ans_pos)

Extract each resident's assignment.

In [6]:
assignment_list(ans_mat, hospital_names, residents)

Bob is assigned to H1
Jim is assigned to H2
Dan is assigned to H1
Sally is assigned to H3
Mary is assigned to H2
